In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
data = pd.read_csv('VIP-X030_1.txt')
data.head()

,VIP'VIP-X030,201907101208,0,000,0.1,000.1,0.2,000.2,0.3,000.3,0.4,0.5,1
0,VIP'VIP-X030,201307121352,88,0,88,100,88,0,88,100,399,35152,1
1,VIP'VIP-X030,201307121353,88,25,88,100,88,0,88,100,175,15418,1
2,VIP'VIP-X030,201307121355,88,125,88,175,88,125,88,125,178,15686,1
3,VIP'VIP-X030,201307121356,88,150,88,200,88,100,88,175,357,31478,1
4,VIP'VIP-X030,201307121357,88,200,88,200,88,150,88,150,47,4143,1


In [3]:
data.shape

(719103, 13)

In [4]:
data.columns = ['Enstruman', 'DateTime', 'O1', 'O2', 'H1', 'H2',
                'L1', 'L2', 'C1', 'C2', 'V1', 'V2', 'Forget']

In [5]:
data['Open'] = data['O1']*1000 + data['O2']
data['High'] = data['H1']*1000 + data['H2']
data['Low'] = data['L1']*1000 + data['L2']
data['Close'] = data['C1']*1000 + data['C2']
data.drop(columns=['O1', 'O2', 'H1', 'H2', 'L1', 'L2', 'C1', 'C2', 'V2',
                   'Forget'], inplace=True)

In [6]:
data.rename(columns={'V1':'Volume'}, inplace=True)
data.drop('Enstruman', axis=1, inplace=True)
data['Quarter'] = data['DateTime'].apply(lambda x: int(x/15))
data.head()

,DateTime,Volume,Open,High,Low,Close,Quarter
0,201307121352,399,88000,88100,88000,88100,13420474756
1,201307121353,175,88025,88100,88000,88100,13420474756
2,201307121355,178,88125,88175,88125,88125,13420474757
3,201307121356,357,88150,88200,88100,88175,13420474757
4,201307121357,47,88200,88200,88150,88150,13420474757


In [7]:
idxOpen = data.groupby('Quarter')['DateTime'].idxmin()
Opens = data.loc[idxOpen, 'Open'].reset_index()
idxClose = data.groupby('Quarter')['DateTime'].idxmax()
Closes = data.loc[idxClose, 'Close'].reset_index()

In [8]:
dataAgg = data.groupby('Quarter').agg({'DateTime':np.min,
                                       'Volume':np.sum,
                                       'High': np.max,
                                       'Low': np.min,
                                      })

In [9]:
dataAgg.reset_index(inplace=True)
dataAgg.head()

,Quarter,DateTime,Volume,High,Low
0,13420474756,201307121352,574,88100,88000
1,13420474757,201307121355,776,88225,88100
2,13420474760,201307121400,2238,88200,88000
3,13420474761,201307121415,6736,88675,88050
4,13420474762,201307121430,4206,88875,88450


In [10]:
dataAgg['Open'] = Opens['Open']
dataAgg['Close'] = Closes['Close']
dataAgg.head()

,Quarter,DateTime,Volume,High,Low,Open,Close
0,13420474756,201307121352,574,88100,88000,88000,88100
1,13420474757,201307121355,776,88225,88100,88125,88175
2,13420474760,201307121400,2238,88200,88000,88175,88150
3,13420474761,201307121415,6736,88675,88050,88150,88475
4,13420474762,201307121430,4206,88875,88450,88450,88700


In [11]:
dataAgg['Date'] = dataAgg['DateTime'].apply(lambda x: int(x/10000))

In [12]:
data2 = dataAgg[(dataAgg['DateTime']> 201900000000) & (dataAgg['DateTime'] <201903010000)]
data2.head()

,Quarter,DateTime,Volume,High,Low,Open,Close,Date
52163,13460068062,201901020930,5768,117700,117300,117700,117450,20190102
52164,13460068063,201901020945,2947,117575,117425,117450,117525,20190102
52165,13460068066,201901021000,3351,117675,117500,117525,117600,20190102
52166,13460068067,201901021005,4993,117650,117400,117600,117400,20190102
52167,13460068068,201901021020,6184,117450,117200,117450,117400,20190102


In [13]:
# EVerything is ok
print(data2[data2['High'] < data2['Close']]['Volume'].sum())
print(data2[data2['High'] < data2['Open']]['Volume'].sum())
print(data2[data2['Low'] > data2['Close']]['Volume'].sum())
print(data2[data2['Low'] > data2['Open']]['Volume'].sum())

0
0
0
0


In [14]:
data2['DateTime'] = data2['DateTime'].apply(lambda x: str(x))
data2.head()

C:\Users\batum\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Quarter,DateTime,Volume,High,Low,Open,Close,Date
52163,13460068062,201901020930,5768,117700,117300,117700,117450,20190102
52164,13460068063,201901020945,2947,117575,117425,117450,117525,20190102
52165,13460068066,201901021000,3351,117675,117500,117525,117600,20190102
52166,13460068067,201901021005,4993,117650,117400,117600,117400,20190102
52167,13460068068,201901021020,6184,117450,117200,117450,117400,20190102


In [15]:
data2['DateTime'] = pd.to_datetime(data2['DateTime'], format='%Y%m%d%H%M' )
data2.head()

C:\Users\batum\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Quarter,DateTime,Volume,High,Low,Open,Close,Date
52163,13460068062,2019-01-02 09:30:00,5768,117700,117300,117700,117450,20190102
52164,13460068063,2019-01-02 09:45:00,2947,117575,117425,117450,117525,20190102
52165,13460068066,2019-01-02 10:00:00,3351,117675,117500,117525,117600,20190102
52166,13460068067,2019-01-02 10:05:00,4993,117650,117400,117600,117400,20190102
52167,13460068068,2019-01-02 10:20:00,6184,117450,117200,117450,117400,20190102


In [16]:
import datetime
data2['Date'] = data2['DateTime'].dt.date

C:\Users\batum\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
data2['Time'] = data2['DateTime'].dt.time
data2.head()

C:\Users\batum\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Quarter,DateTime,Volume,High,Low,Open,Close,Date,Time
52163,13460068062,2019-01-02 09:30:00,5768,117700,117300,117700,117450,2019-01-02,09:30:00
52164,13460068063,2019-01-02 09:45:00,2947,117575,117425,117450,117525,2019-01-02,09:45:00
52165,13460068066,2019-01-02 10:00:00,3351,117675,117500,117525,117600,2019-01-02,10:00:00
52166,13460068067,2019-01-02 10:05:00,4993,117650,117400,117600,117400,2019-01-02,10:05:00
52167,13460068068,2019-01-02 10:20:00,6184,117450,117200,117450,117400,2019-01-02,10:20:00


In [18]:
data2['High'] = data2['High'] / 1000
data2['Open'] = data2['Open'] / 1000
data2['Low'] = data2['Low'] / 1000
data2['Close'] = data2['Close'] / 1000

C:\Users\batum\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\batum\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\batum\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [19]:
data2.shape

(1751, 9)

In [20]:
data2.head()

,Quarter,DateTime,Volume,High,Low,Open,Close,Date,Time
52163,13460068062,2019-01-02 09:30:00,5768,117.700,117.300,117.700,117.450,2019-01-02,09:30:00
52164,13460068063,2019-01-02 09:45:00,2947,117.575,117.425,117.450,117.525,2019-01-02,09:45:00
52165,13460068066,2019-01-02 10:00:00,3351,117.675,117.500,117.525,117.600,2019-01-02,10:00:00
52166,13460068067,2019-01-02 10:05:00,4993,117.650,117.400,117.600,117.400,2019-01-02,10:05:00
52167,13460068068,2019-01-02 10:20:00,6184,117.450,117.200,117.450,117.400,2019-01-02,10:20:00


In [21]:
#data2.to_csv('VIOP.csv')